In [0]:
import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, TimeDistributed
from keras.layers import Conv2D, MaxPooling2D, LSTM

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv("drive/My Drive/JS_peaks.csv", index_col= "DateTime")
data.index = pd.to_datetime(data.index)

In [0]:
df = data["2019"].drop("peaks", axis=1)

In [0]:
peek_future = 1
df["next_price"] = df.close.shift(-peek_future)
df = df[:-peek_future]
X = df.drop("next_price", axis=1)
y = df.next_price

In [0]:
look_back_window = 15
seq_X = []
seq_y = []
X = X.values
y = y.values
for i in range(look_back_window,len(X)):
  seq_X.append(X[i-look_back_window:i])
  seq_y.append(y[i])

seq_X = np.array(seq_X)
seq_y = np.array(seq_y)

In [36]:
seq_X.shape

(425560, 15, 29)

In [0]:
train_size = 424000
num_rows, num_cols = seq_X.shape[1], seq_X.shape[2]

input_shape = (seq_X.shape[1], seq_X.shape[2],1)


X_train = seq_X[:train_size]
y_train = seq_y[:train_size]

X_test = seq_X[train_size:]
y_test= seq_y[train_size:]

X_train1 = X_train.reshape(X_train.shape[0], num_rows, num_cols, 1)
X_test1 = X_test.reshape(X_test.shape[0], num_rows, num_cols, 1)


In [0]:
model = Sequential()
model.add(TimeDistributed(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=input_shape)))
model.add(TimeDistributed(Conv2D(256, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [47]:
model.fit(X_train1, y_train,
          batch_size=32,
          epochs = 2,
          verbose=0)
         #validation_data=(X_test1, y_test))

ValueError: ignored